# Essential workers testing for SF
--------------------------------------

This notebook lists different test cases and results for SF data.

Including: 
* **different types of essential workers:** edu, office, social, supermarket
        for example: 100% of edu workers comply vs. the same number of random individuals comply
                     100% of office workers comply vs. the same number of random individuals comply etc.
* **different beta values:** 0.1~0.4
* **different ratios of (# of essential population)/(# of total population)**
        for example: 10% of the total population are essential vs. 50% of the total population are essential workers

## Contents

1. Model parameter settings:
        1.1 Parameters that are the same for all simulations
        1.2 Parameters to change:
            1.2.1 Types of essential workers
            1.2.2 Ratio of essential population
            1.2.3 Essential mobility pattern
2. Instructions to run the codes
3. Results and figures
4. Appendix: How work places are assigned to essential workers
----------------------------------
----------------------------------

## 1. Model parameter settings

### 1.1 For all cases, the followings are the same:
-------------------------

* **downsampling rate:** 100

* **Population and sites details after downsampling:**

        The total population:8499 ;  The total number of sites:75

        Population in each age group:
        [387,   592,   308,    526,    3293,   1671,   1365,   362]
        [0-5,   5-14,  15-19,  20-24,  25-44,  45-59,  60-79,  80+]

        Number of sites in each type:
        [5,    37,      28,      5]
        [edu,  office,  social,  supermarket]


* **The mob_rate for regular individuals:**
        mob_rate_per_age_per_type = [
               [5,    0,    0,   0],    # 0-5
               [5,    0,    0,   0],    # 5-14
               [5,    0,    3.6, 0.22], # 15-19
               [1.48, 3.52, 3.6, 0.21], # 20-24
               [0,    5,    3.6, 0.27], # 25-44
               [0,    5,    3.6, 0.36], # 45-59
               [0,    0,    3.6, 0.35], # 60-79
               [0,    0,    3.6, 0.35]] # 80+

* **Duration time at each type of site:** dur_mean_per_type = [5.0, 5.0, 0.64, 0.4]


* **Variety per type:**   variety_per_type = [1, 1, 10, 2]


* **Essential proportion in each age group:**
        essential_distribution:
        [0,     0,     0.04,   0.06,   0.55,   0.3,    0.05,   0]
        [0-5,   5-14,  15-19,  20-24,  25-44,  45-59,  60-79,  80+]
        for example, 4% of the population in age group 15-19 are essential workers
        
* **Simulation period:**
        start_date = 2020-03-04
        end_date = 2020-06-19

* **random repeats:** 40

* **testing parameters:**
        testing freq:     24 hrs
        reperting lag:    48 hrs
        testing capacity per batch: 1
        test false rate:  0
        test smart contacts: 25
        isolation after tested positive: 7*24 hrs
        testing policy: basic
        

---------------------------------------------------
### 1.2 Parameters to change

###   1.2.1 Types of essential workers
--------------------------------

In the codes, **essential_type** denotes different types of essential workers.

Since there are 4 types of sites, there will be 4 types of essential workers:

            essential_type = 0  →  edu workers (e.g. teachers)
            essential_type = 1  →  office workers
            essential_type = 2  →  social places workers (e.g. restuarants)
            essential_type = 3  →  supermarket workers

### 1.2.2 Ratio of essential population
------------------

In the codes, **essential_to_total_ratio_list** denotes ratio of essential workers.

You can either input one number or a list of ratios.

For example, if the ratio is 0.1, it means 10% of the total population are essential workers. Then this 10% will be distributed into different age groups at a possibility, essential_distribution, specified in the previous section.

An example of the # essential workers distribution with 10% of 8499 (previous section) is:

        num_essential_per_age_group = 
        [0,    0,     34,     51,     467,    255,    42,     0]      total number: 849, actual ratio: 10.11%
        [0-5,  5-14,  15-19,  20-24,  25-44,  45-59,  60-79,  80+]


### 1.2.3 Essential mobility patterns
----------------

The mobility patterns for essential workers differ from regular individuals in the following 3 ways:
* mob_rate: freq of visits to different sites
* dur_time: how long they stay for each visit
* work_place: essential workers always visit one specific work place while rugelar individuals don't

First, an essential type should be specified. The mob rate and duration time for essential workers are based on this essential type. How we assign a work place for each essential worker will be addressed in the appendix.

Mobility rates for different essential_types:

      essential_type = 0  →  edu workers             →     [5, 0, 3.6, 0.27]
      essential_type = 1  →  office workers          →     [0, 5, 3.6, 0.27]
      essential_type = 2  →  social places workers   →     [0, 0, 5,   0.27]
      essential_type = 3  →  supermarket workers     →     [0, 0, 3.6, 5   ]
      e.g. for each social place worker, he/she visits a social place 5 times a week, supermarkets 0.27 times a week
            

Duration time for different essential_types:

      essential_type = 0  →  edu workers             →     [5, 0, 0.64, 0.4]
      essential_type = 1  →  office workers          →     [0, 5, 0.64, 0.4]
      essential_type = 2  →  social places workers   →     [0, 0, 5,    0.4]
      essential_type = 3  →  supermarket workers     →     [0, 0, 0.64, 5  ]
      e.g. for each social place worker, he/she spends 5 hrs at a social place each visit, 0.4 hrs at supermarkets
            
**The only parameter needs to be changed manually is essential_type**, mob_rate and dur_time for essential workers will be assigned automatically.

-----------------------
## 2. Instructions to run simulations

1. **generate town data:**  
        run this on local computer: emma_exp_one_comp_notebook.ipynb
        parameters that one can change:
            * essential_to_total_ratio_list (line 50)
            * essential_type (line 55)
2. **upload the town data .pk file to the cluster:**
        an example of the naming: sf_type3_prop0.1_ds100.pk
        it stands for: SF data, essential type 3 (supermarket), 10% of total population are workers, downsampled by 100
3. **run the simulations on cluster:**
        run this: emma_experiment_no_towngen.py
            should change the town data file name before running (line 61)
            e.g. mob_settings = 'sf_type3_prop0.1_ds100.pk'
        parameters that one can change:
            * beta value (line 89-98)
            * compliance rate (line 210)
4. **download the results .pk file to local computer:**
        an example of the naming: comp[1.0]_repeats40_sf_type3_prop0.1_ds100.pk
        it stands for essential compliance rate 100%, simulation repeated 40 times
        (note that random individual compliance rate is 100%*essential_to_total_ratio_list so that the number of people comply are approximately the same)
5. **go to the results section (next) and plot:**
        change the file name and also label names if neccessary.

---------------------------------------
## 3. Results

In [ ]:
# run this before plotting

%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')


import pandas as pd
import numpy as np
import networkx as nx
import copy
import scipy as sp
import math
import seaborn
import pickle
import warnings
import matplotlib
import re
import multiprocessing

from lib.mobilitysim import MobilitySimulator
from lib.dynamics import DiseaseModel
from lib.inference import * 
from bayes_opt import BayesianOptimization
from lib.parallel import *
from lib.distributions import CovidDistributions
from lib.plot import Plotter
from lib.data import collect_data_from_df
from lib.measures import (
    MeasureList, 
    BetaMultiplierMeasure, 
    BetaMultiplierMeasureByType,
    SocialDistancingForAllMeasure, 
    SocialDistancingForKGroups,
    SocialDistancingByAgeMeasure,
    SocialDistancingForPositiveMeasure, 
    ComplianceForAllMeasure,
    Interval,
    ComplianceForEssentialWorkers)

from lib.mobilitysim import MobilitySimulator
from lib.town_data import generate_population, generate_sites, compute_distances
from lib.town_maps import MapIllustrator


def save_summary(summary, filename):
    with open('summaries/' + filename, 'wb') as fp:
        pickle.dump(summary, fp)
    
def load_summary(filename):
    with open('summaries/' + filename, 'rb') as fp:
        summary = pickle.load(fp)
    return summary

### Edu essential workers: teachers (type 0)

Parameters:
        
        essential_to_total_ratio_list = 0.1
        essential_type = 0
        beta = household: 0, elsewhere: 0.1
        p_compliance = 1.0
        
        school ID               = [9,   57,  41,  71,  28]
        teachers at each school = [164, 167, 170, 184, 174]
Compare the total infections for **100% of the teachers** comply versus **10% of random individuals** comply.

In [ ]:
summaries_SD_6 = load_summary('comp[1.0]_repeats40_sf_essential-type2_prop0.1_ds100.pk')
FIGSIZE = (8, 4)
plotter = Plotter()
title = ['random individuals','essential workers']
summaries_ = summaries_SD_6['basic']
c = 0
runstr = f'run{c}_'

plotter.compare_total_infections(
    summaries_, 
    titles=title, 
    figtitle=r'Total Infections with Contact Tracing Case Isolation for All',
    filename=runstr + 'ST_all', 
    figsize=FIGSIZE, acc=500, 
    ymax=2000, errorevery=14)

## 4 Appendix: How work places are assigned to essential workers
-----------------------------

Once the essential type and essential population are specified, we will assign a work place for each essential worker through the following:

(e.g. essential type 3: supermarket)
1. Collect the site ID of all supermarkets in the site pool and save as work_site for future.
        e.g. work_site=[1, 49, 33, 68, 10]. 
        There are 5 supermarkets, they are site #1, #49, #33, #68, #10 respectively.
2. We assume that each work site requires the same number of workers, with some stochasticity here. At this step, we generate a probability array for all workers being assigned to each supermarket.
        e.g. prob_essential_workers_per_site = [0.19091967, 0.19441211, 0.19790454, 0.21420256, 0.20256112]
        For one specific worker, there's 19.1% chance he/she works at site #1, 19.4% chance at site #49 and so forth.
3. Then we will loop over all essential workers, and for each one, we will assign him/her a work place ID based on the probabilities at step 2. This ID will stay with this worker throughout the simulation.
        e.g. numbers of essential workers at the each of the 5 supermarkets are
             [164, 167, 170, 184, 174]
This step is already incorprated in the town_data.py, and will automatically adjust according to the essential type.**Therefore will not need to modify for different runs.**